In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics.scorer import make_scorer

In [18]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [19]:
from preprocess import preprocess_train, preprocess, FEATURES, CATEGORICAL_FEATURES, TEST_FEATURES, CATEGORICAL_TEST_FEATURES_IDX

In [20]:
#train[u'Входные группы']

In [21]:
X, y = preprocess_train(train, categotical_features=CATEGORICAL_FEATURES, features=TEST_FEATURES)

In [22]:
X, y = shuffle(X, y)

In [23]:
y.shape

(8716,)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75)

## CatBoost

In [25]:
# Training simple model
from catboost import CatBoostRegressor

In [33]:
cbr = CatBoostRegressor(iterations=2000, logging_level='Silent')

In [34]:
scorer = make_scorer(lambda a, b: mean_squared_error(a, b)**.5)
scores = cross_val_score(cbr, X, y, cv=5, scoring=scorer)

In [35]:
np.mean(scores)

245.38473039450932

In [36]:
y_pred = cbr.fit(X_train, y_train, cat_features=CATEGORICAL_TEST_FEATURES_IDX).predict(X_test)
print "Mean squared error: {}".format(mean_squared_error(y_test, y_pred)**.5)

Mean squared error: 284.83019248


In [37]:
# Make submission
cbr.fit(X, y, cat_features=CATEGORICAL_TEST_FEATURES_IDX)

In [38]:
test = preprocess(test, CATEGORICAL_FEATURES)

In [39]:
X = test[TEST_FEATURES].values

In [40]:
predictions = cbr.predict(X)

In [41]:
predictions

array([ 635.54991182,  917.26257847,  318.36459337, ...,  243.35078905,
        -88.95946039,   14.09377676])

In [42]:
def make_submission(ids, predictions):
    df = pd.concat([ids, pd.Series(predictions)], axis=1)
    return df.rename(columns={0: 'value'})

In [43]:
l = test.id

In [44]:
df = pd.concat([test.id, pd.Series(predictions)], axis=1)

In [45]:
df = df.rename(columns={0: 'value'})

In [46]:
df.to_csv("cat_baseline.csv", index=False)